In [3]:
import pandas as pd
import numpy as np
from queryrunner_client import Client
from tqdm import tqdm,tqdm_notebook
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [4]:
email = 'rajansharma@uber.com'
qr = Client(user_email = email)

In [5]:
query_op = qr.execute_report('1gDogABIR',datacenter = 'dca1', timeout = 3600)
df = pd.DataFrame(query_op.load_data(), columns = query_op.columns)

04/27/2021 04:33:41 PM  Fetching metadata for Report 1gDogABIR 
2021-04-27 16:33:41,405 querybuilder_client INFO  Fetching metadata for Report 1gDogABIR 
04/27/2021 04:33:41 PM  Loaded object metadata. 
2021-04-27 16:33:41,456 querybuilder_client INFO  Loaded object metadata. 
04/27/2021 04:33:41 PM  You're using Queryrunner V2! 
04/27/2021 04:33:41 PM  [Polling] 7e088323-0119-49ba-b0f7-686380c26ab8 
04/27/2021 04:33:41 PM  [Status] created 
04/27/2021 04:33:42 PM  [Status] started waiting to execute 
04/27/2021 04:33:47 PM  [Status] started execution 
04/27/2021 04:35:47 PM  [Status] completed success 
04/27/2021 04:35:47 PM  [Query Success] completed success 


In [6]:
# df.createddate = pd.to_datetime(df.createddate).dt.date

In [7]:
df.head(2)

,id,name,shippingaddress,createddate,service_country__c,rank_a,shippinglatitude,shippinglongitude
0,0011J00001mMqXsQAK,L'Ami du Pain,Saint-Jean-de-BrayeFranceAddress1.95381458...,2021-04-08,France,1,47.908730,1.953810
1,0011J00001mypd8QAA,Pizza kebap Reconvilier,ReconvilierSwitzerlandAddress7.2271822732...,2021-03-16,Switzerland,1,47.234858,7.227182


In [8]:
# df.createddate = pd.to_datetime(df['createddate']).dt.normalize()

In [9]:
sf_report = df.copy()

In [10]:
sf_report.shape

(1325, 8)

In [160]:
# 4/12/2021
# 4/1/2021
# 3/1/2021
# 4/2/2021
# 4/11/2021

df = sf_report[(sf_report.createddate == '2021-04-11')]

In [175]:
query_op = qr.execute_report('Duhbo5Ndv', datacenter = 'dca1', parameters = {'pass_date':'2021-03-30'})
master = pd.DataFrame(query_op.load_data(), columns = query_op.columns)

04/27/2021 07:49:59 PM  Fetching metadata for Report Duhbo5Ndv 
2021-04-27 19:49:59,792 querybuilder_client INFO  Fetching metadata for Report Duhbo5Ndv 
04/27/2021 07:49:59 PM  Loaded object metadata. 
2021-04-27 19:49:59,837 querybuilder_client INFO  Loaded object metadata. 
04/27/2021 07:49:59 PM  Templating query for report Duhbo5Ndv 
2021-04-27 19:49:59,841 querybuilder_client INFO  Templating query for report Duhbo5Ndv 
04/27/2021 07:49:59 PM  Templated query successfully. 
2021-04-27 19:49:59,884 querybuilder_client INFO  Templated query successfully. 
04/27/2021 07:49:59 PM  You're using Queryrunner V2! 
04/27/2021 07:49:59 PM  [Polling] ed3614d3-c6f0-4651-9cdb-c0c3dd9517d8 
04/27/2021 07:50:00 PM  [Status] created 
04/27/2021 07:50:01 PM  [Status] started execution 
04/27/2021 07:50:09 PM  [Status] completed success 
04/27/2021 07:50:09 PM  [Query Success] completed success 


In [176]:
master.head(2)

,id,name,shippingaddress,createddate,service_country__c,rank_a,shippinglatitude,shippinglongitude
0,0011J000018UihGQAS,5 Pailles,ParisFrance\N\N75010\N79 Rue du Faubourg...,2017-07-04T14:06:40.000+0000,France,1,NaN,NaN
1,0011J000018j6fSQAQ,Chiquitin - Luz Verde,ParisFranceZip2.3378575009Ile-de-France2...,2017-08-03T12:17:24.000+0000,France,1,48.87704,2.33785


In [177]:
df.shape,master.shape

((7, 8), (52446, 8))

In [164]:
df.head(2)

,id,name,shippingaddress,createddate,service_country__c,rank_a,shippinglatitude,shippinglongitude
296,0011J00001oBaaWQAS,Wiedikon point,ZurichSwitzerlandAddress8.5229918003ZHZu...,2021-04-11,Switzerland,1,47.367394,8.522991
523,0011J00001oBaZ9QAK,Juliacha,ZúrichSwitzerlandBlock8.560753334325058803...,2021-04-11,Switzerland,1,47.367487,8.560753


In [165]:
master = master[~master['shippinglongitude'].isnull()]
df = df[~df['shippinglongitude'].isnull()]

In [166]:
df.shape,master.shape

((7, 8), (339251, 8))

In [167]:
list_of_dicts = []

# Distance & Fuzzy Match

In [168]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km
# Considering earth spherical radius as 6373 in kms, Multiply the result with 6373 to get the distance in KMS.
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    
    return d

In [169]:
for row in tqdm(list(master.itertuples())):
    name = row.name
    uuid = row.id
    lat, long = row.shippinglatitude, row.shippinglongitude
    country = row.service_country__c
    samp_sub = df[df.service_country__c==country]
    for row2 in samp_sub.itertuples():
        distance_val = distance([float(lat),float(long)], [float(row2.shippinglatitude), float(row2.shippinglongitude)])
        if distance_val <= 0.3:
            row_dict = {'id':uuid,'acc_name':name, 'shippinglatitude':lat, 'shippinglongitude':long, 
                        'service_country__c':country, 'created_date':row2.createddate, 'sales_uuid':row2.id,
                        'sales_acc_name':row2.name, 'sales_country':row2.service_country__c, 'sales_latitude':row2.shippinglatitude,
                        'sales_longitude':row2.shippinglongitude, 'distance':distance_val}
            list_of_dicts.append(row_dict)

100%|██████████| 339251/339251 [12:15<00:00, 461.42it/s]


In [170]:
df_all = pd.DataFrame(list_of_dicts)

In [171]:
df_all['ratio'] = df_all.apply(lambda x: fuzz.ratio(x['acc_name'], x['sales_acc_name']), axis=1)
df_all['partial'] = df_all.apply(lambda x: fuzz.partial_ratio(x['acc_name'], x['sales_acc_name']), axis=1)
df_all['token'] = df_all.apply(lambda x: fuzz.token_sort_ratio(x['acc_name'], x['sales_acc_name']), axis=1)

In [172]:
final_df = df_all[['id','acc_name','service_country__c','shippinglatitude','shippinglongitude',
        'created_date','sales_uuid','sales_acc_name','sales_country','sales_latitude','sales_longitude',
       'distance','partial','token','ratio']]

In [173]:
final_df

,id,acc_name,service_country__c,shippinglatitude,shippinglongitude,created_date,sales_uuid,sales_acc_name,sales_country,sales_latitude,sales_longitude,distance,partial,token,ratio
0,0011J00001RIXChQAP,Pizza & Friends,Switzerland,47.371166,8.528789,2021-04-11,0011J00001oBabtQAC,Cafe Gusto,Switzerland,47.369513,8.526863,0.234140,20,26,24
1,0011J00001Rgg1jQAB,Viêt-Long,Switzerland,47.493268,8.742863,2021-04-11,0011J00001oBaVLQA0,Konak Food,Switzerland,47.493018,8.743344,0.045593,22,22,21
2,0011J00001RIVDaQAP,Restaurant Al Ponte,Switzerland,47.384846,8.508464,2021-04-11,0011J00001oBadzQAC,Aria Laden,Switzerland,47.384007,8.508416,0.093363,30,34,28
3,0011J00001RHyNFQA1,Kramer Gastronomie,Switzerland,47.383090,8.506001,2021-04-11,0011J00001oBadzQAC,Aria Laden,Switzerland,47.384007,8.508416,0.208457,40,43,36
4,0011J00001RJ94kQAD,Al 21,Switzerland,47.368177,8.522098,2021-04-11,0011J00001oBaaWQAS,Wiedikon point,Switzerland,47.367394,8.522991,0.110015,20,11,11
5,0011J00001lXHMSQA4,The Bite Gourmet Burgers,Switzerland,47.384846,8.508464,2021-04-11,0011J00001oBadzQAC,Aria Laden,Switzerland,47.384007,8.508416,0.093363,20,18,18
6,0011J00001RhQ1VQAV,Justus Laden & Lokal,Switzerland,47.367319,8.560895,2021-04-11,0011J00001oBaZ9QAK,Juliacha,Switzerland,47.367487,8.560753,0.021531,25,38,29
7,0011J00001lVrP8QAK,Freyas Sommer,Switzerland,47.371543,8.524693,2021-04-11,0011J00001oBabtQAC,Cafe Gusto,Switzerland,47.369513,8.526863,0.278655,30,43,26
8,0011J00001RJfMHQA1,Burgstein`s Gasthaus Penalty,Switzerland,47.371461,8.528520,2021-04-11,0011J00001oBabtQAC,Cafe Gusto,Switzerland,47.369513,8.526863,0.249980,40,32,32
9,0011J00001ky3GcQAI,Ottos Oats,Switzerland,47.369571,8.526614,2021-04-11,0011J00001oBabtQAC,Cafe Gusto,Switzerland,47.369513,8.526863,0.019830,20,40,20


In [174]:
final_df.to_csv("fbs_apr11.csv", index=False, encoding='UTF-8')

In [33]:
# def fuzz_partial(match,compare):
#     partial = fuzz.partial_ratio(match,compare)
#     token = fuzz.token_sort_ratio(match,compare)
#     return [partial,token]

In [41]:
# for row in tqdm(list(df_all.itertuples())):
#     name = row.acc_name
#     uuid = row.id
#     lat,long = row.shippinglatitude , row.shippinglongitude
#     for row2 in df_all.itertuples():
#         fuzz_partial_score = fuzz_partial(name, row2.sales_acc_name)
#         row_dict = {'Acc_ID':uuid,'Acc_Name':name, 'Acc_lat':lat, 'Acc_long':long,
#                     'Account[By Sales]':row2.sales_acc_name, 'Acc_lat[Sales]':row2.sales_latitude, 'Acc_long[Sales]':row2.sales_longitude,
#                     'partial':fuzz_partial_score[0],'token':fuzz_partial_score[1]}
#         list_of_dicts.append(row_dict)

In [21]:
final_df.shape

(10101, 14)